In [1]:
import random
import warnings
import numpy as np
import scipy.io as sio
import tensorflow as tf
from keras.models import Model#Sequential, Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
#from keras.optimizers import SGD
from keras.layers.convolutional import MaxPooling2D, Conv2D
#from keras.utils import np_utils
from keras.layers.normalization import BatchNormalization
from keras.layers import Activation, Conv2DTranspose, Input, concatenate, Cropping2D
from keras.layers.merge import concatenate
from keras import backend as K
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model, load_model
from skimage.io import imread, imshow, imread_collection, concatenate_images
from keras import optimizers

from ParamConfig import *
from PathConfig import *
from LibConfig import *


/home/jonas/anaconda3/envs/py3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jonas/anaconda3/envs/py3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jonas/anaconda3/envs/py3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jonas/anaconda3/envs/py3/lib/python3.7/site-packages

In [2]:
X_train,Y_train  = DataLoad_Train(train_size=190,train_data_dir=train_data_dir, \
                                                                 data_dim=DataDim,in_channels=Inchannels, \
                                                                 model_dim=ModelDim,data_dsp_blk=data_dsp_blk, \
                                                                 label_dsp_blk=label_dsp_blk,start=1, \
                                                                 datafilename=datafilename,dataname=dataname, \
                                                                 truthfilename=truthfilename,truthname=truthname, folder_dataset=folder_dataset)
# Change data type (numpy --> tensor)


X_test,Y_test  = DataLoad_Test(train_size=TrainSize,train_data_dir=train_data_dir, \
                                                                 data_dim=DataDim,in_channels=Inchannels, \
                                                                 model_dim=ModelDim,data_dsp_blk=data_dsp_blk, \
                                                                 label_dsp_blk=label_dsp_blk,start=1, \
                                                                 datafilename=datafilename,dataname=dataname, \
                                                                 truthfilename=truthfilename,truthname=truthname, folder_dataset=folder_dataset)
# Change data type (numpy --> tensor)

georec1
vmodel1
georec2


/home/jonas/anaconda3/envs/py3/lib/python3.7/site-packages/skimage/util/shape.py:93: RuntimeWarning: Cannot provide views on a non-contiguous input array without copying.
  warn(RuntimeWarning("Cannot provide views on a non-contiguous input "


vmodel2
georec3
vmodel3
georec4
vmodel4
georec5
vmodel5
georec6
vmodel6
georec7
vmodel7
georec8
vmodel8
georec9
vmodel9
georec10
vmodel10
georec11
vmodel11
georec12
vmodel12
georec13
vmodel13
georec14
vmodel14
georec15
vmodel15
georec16
vmodel16
georec17
vmodel17
georec18
vmodel18
georec19
vmodel19
georec20
vmodel20
georec21
vmodel21
georec22
vmodel22
georec23
vmodel23
georec24
vmodel24
georec25
vmodel25
georec26
vmodel26
georec27
vmodel27
georec28
vmodel28
georec29
vmodel29
georec30
vmodel30
georec31
vmodel31
georec32
vmodel32
georec33
vmodel33
georec34
vmodel34
georec35
vmodel35
georec36
vmodel36
georec37
vmodel37
georec38
vmodel38
georec39
vmodel39
georec40
vmodel40
georec41
vmodel41
georec42
vmodel42
georec43
vmodel43
georec44
vmodel44
georec45
vmodel45
georec46
vmodel46
georec47
vmodel47
georec48
vmodel48
georec49
vmodel49
georec50
vmodel50
georec51
vmodel51
georec52
vmodel52
georec53
vmodel53
georec54
vmodel54
georec55
vmodel55
georec56
vmodel56
georec57
vmodel57
georec58
vmodel5

In [4]:
linhas=X_train.shape[1]
colunas= X_train.shape[2]
canais = X_train.shape[3]


def apply_maxPooling(camada):
    camada = MaxPooling2D(pool_size=(2,2),padding='same') (camada)
    return camada

def reduzir(camada,out_size):
    camada = Conv2D(filters=out_size, kernel_size=(3, 3), input_shape=(linhas, colunas, canais), strides=1, padding='same') (camada)
    camada = BatchNormalization() (camada)
    camada = Activation('relu') (camada)
    
    #reaplicando novamente no dado
    camada = Conv2D(filters=out_size, kernel_size=(3, 3), input_shape=(linhas, colunas, canais), strides=1, padding='same') (camada)
    camada = BatchNormalization() (camada)
    camada = Activation('relu') (camada)
    
    return camada


def reduzir_last(camada,out_size):
    camada = Conv2D(filters=out_size, kernel_size=(3, 3), input_shape=(linhas, colunas, canais), strides=1, padding='same') (camada)
    camada = BatchNormalization() (camada)
    camada = Activation('relu') (camada)

    return camada

def aumentar_last(camada_corrente,camada_de_reducao_correspondente,out_size):
    camada = Conv2DTranspose(filters=out_size,kernel_size=(2,2),strides=2,padding='same') (camada_corrente)
    camada = concatenate([camada,camada_de_reducao_correspondente],axis=3) #aqui concateno o dado após a deconvolução com o dado correspondente na camada de redução
    camada = reduzir_last(camada,out_size)

    return camada


def aumentar(camada_corrente,camada_de_reducao_correspondente,out_size):
    camada = Conv2DTranspose(filters=out_size,kernel_size=(2,2),strides=2,padding='same') (camada_corrente)
    camada = concatenate([camada,camada_de_reducao_correspondente],axis=3) #aqui concateno o dado após a deconvolução com o dado correspondente na camada de redução
    camada = reduzir(camada,out_size)

    return camada


def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)


#sgd = optimizers.SGD(lr=0.01, clipnorm=1.)


def create_model(linhas,colunas,canais):
    inputs = Input((linhas,colunas,canais))
    
    filtros=[64,128,256,512,1024]
    c1 = reduzir(inputs,filtros[0])
    c2 = reduzir(apply_maxPooling(c1),filtros[1])
    c3 = reduzir(apply_maxPooling(c2),filtros[2])
    c4 = reduzir(apply_maxPooling(c3),filtros[3])
    c5 = reduzir(apply_maxPooling(c4),filtros[4])
    

    c6 = aumentar(c5,c4,filtros[3])
    c7 = aumentar(c6,c3,filtros[2])
    c8 = aumentar(c7,c2,filtros[1])
    c9 = aumentar_last(c8,c1,filtros[0])
    
    c9 = Cropping2D(cropping=(99, 1)) (c9) #fiz um crop na imagem que era (400,304) e deixei com (202,302)
    c9 = reduzir_last(c9,filtros[0])
    c9 = Conv2D(filters=1, kernel_size=(1, 1), strides=1) (c9)

    model = Model(inputs=[inputs],outputs=[c9])
    model.compile(optimizer='adam', loss='mse', metrics=['mse'])
    model.summary()
    return model

model = create_model(linhas,colunas,canais)


Instructions for updating:
Colocations handled automatically by placer.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 400, 304, 29) 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 400, 304, 64) 16768       input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 400, 304, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 400, 304, 64) 0           batch_normalization_1[0][0]      
____________________

In [7]:
# Fit model
#from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


#earlystopper = EarlyStopping(patience=10, verbose=1)
#checkpointer = ModelCheckpoint('model-dsbowl2018-1.h5', verbose=1, save_best_only=True)
#ReduceLROnPlateau = ReduceLROnPlateau(factor=0.1, patience=3, min_lr=0.00001, verbose=1)
#model.fit(X_train, Y_train, validation_split=0.1, batch_size=5, epochs=10, 
#                    callbacks=[earlystopper])

model.fit(X_train, Y_train, validation_split=0.1, batch_size=5, epochs=100)

Instructions for updating:
Use tf.cast instead.
Train on 171 samples, validate on 19 samples
Epoch 1/100
171/171 [==============================] - 36s 210ms/step - loss: 2.8613 - mse: 2.8613 - val_loss: 1467759.7171 - val_mse: 1467759.7500
Epoch 2/100
171/171 [==============================] - 24s 139ms/step - loss: 0.1805 - mse: 0.1805 - val_loss: 51.3935 - val_mse: 51.3935
Epoch 3/100
171/171 [==============================] - 24s 138ms/step - loss: 0.0605 - mse: 0.0605 - val_loss: 2.7235 - val_mse: 2.7235
Epoch 4/100
171/171 [==============================] - 23s 136ms/step - loss: 0.0449 - mse: 0.0449 - val_loss: 1.6757 - val_mse: 1.6757
Epoch 5/100
171/171 [==============================] - 23s 137ms/step - loss: 0.0397 - mse: 0.0397 - val_loss: 1.1128 - val_mse: 1.1128
Epoch 6/100
171/171 [==============================] - 23s 136ms/step - loss: 0.0385 - mse: 0.0385 - val_loss: 0.8258 - val_mse: 0.8258
Epoch 7/100
171/171 [==============================] - 24s 139ms/step - loss:

171/171 [==============================] - 24s 138ms/step - loss: 0.0124 - mse: 0.0124 - val_loss: 0.0137 - val_mse: 0.0137
Epoch 59/100
171/171 [==============================] - 23s 136ms/step - loss: 0.0122 - mse: 0.0122 - val_loss: 0.0218 - val_mse: 0.0218
Epoch 60/100
171/171 [==============================] - 23s 136ms/step - loss: 0.0119 - mse: 0.0119 - val_loss: 0.0183 - val_mse: 0.0183
Epoch 61/100
171/171 [==============================] - 24s 138ms/step - loss: 0.0118 - mse: 0.0118 - val_loss: 0.0165 - val_mse: 0.0165
Epoch 62/100
171/171 [==============================] - 23s 137ms/step - loss: 0.0115 - mse: 0.0115 - val_loss: 0.0130 - val_mse: 0.0130
Epoch 63/100
171/171 [==============================] - 24s 138ms/step - loss: 0.0123 - mse: 0.0123 - val_loss: 0.0146 - val_mse: 0.0146
Epoch 64/100
171/171 [==============================] - 24s 139ms/step - loss: 0.0118 - mse: 0.0118 - val_loss: 0.0145 - val_mse: 0.0145
Epoch 65/100
171/171 [==============================] 

In [8]:

# from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
# callbacks = [
#     EarlyStopping(patience=10, verbose=1),
#     ReduceLROnPlateau(factor=0.1, patience=3, min_lr=0.001, verbose=1),
#     ModelCheckpoint('model-tgs-salt.h5', verbose=1, save_best_only=True, save_weights_only=True)
# ]
# results = model.fit(X_train, Y_train, batch_size=5, epochs=50, callbacks=callbacks)

In [11]:
preds_test = model.predict(X_test)

In [24]:
def generate_predicoes(predicoes,Y_test):
    for i in range(0,predicoes.shape[0]):
        print ('Salvando modelo '+ str(i+1))
        #imprimindo o modelo de velocidade real
        vmodel = Y_test[i];
        vmodel = vmodel.reshape(202,302)
        sio.savemat('vmodel'+str(i)+'.mat',{'vmodel': vmodel})
        
        #imprimindo o modelo de velocidade predito
        vmodel = predicoes[i];
        vmodel = vmodel.reshape(202,302)
        sio.savemat('predito'+str(i)+'.mat',{'vmodel': vmodel})

generate_predicoes(preds_test,Y_test)

Salvando modelo 1
Salvando modelo 2
Salvando modelo 3
Salvando modelo 4
Salvando modelo 5
Salvando modelo 6
Salvando modelo 7
Salvando modelo 8
Salvando modelo 9
Salvando modelo 10
